# Unit 14 Case Study - Analyzing Airline Flight Delays
___Zach Brown___  
___August 9, 2017___

## Introduction

In the age of big data, the possibilities for analysis have been greatly expanded.  However, with new possibilites also come new challenges.  While many organizations now have massive data sets with the possibility to generate useful insights, the size of these data sets can also be unwieldy.  

In this case study, we will examine a data set containing domestic flight data for U.S. airlines for the years 1987 through 2008.  Using standard python and pandas operations, all of the data is loaded into memory at once.  However, the uncompressed data files for this data set are between 450 and 700 megabytes per year for a total of 11.2 gigabytes.  That is simply too much data to be loaded into memory at one time using standard hardware at the current time.  A different method will need to be utilized.

In order to analyze the airline data set, we will use the GraphLab library from Turi (formerly Dato).  GraphLab is a parallel programming framework written in C++ and will allow us to use the split-apply-combine method of operating on this large dataset.  In order to test this functionality, we will explore the data specific to flight delays.  We will examine number of delays broken down by airport and flight route and then attempt to determine the length of a delay using linear regression.

## Methods and Results

First we need to create a directory to hold the data files if it does not already exist

In [ ]:
import os, sys
# create a Data directory if not done already
path = "Data"
if not os.path.exists(path):
    os.mkdir(path)

Now that the Data directory exists, each of the compressed files for the years 1987 through 2008 can be downloaded.

In [ ]:
import urllib.request

years_to_download = range(1987,2009) # get the years 1987 through 2008
baseurl = 'http://stat-computing.org/dataexpo/2009/%d.csv.bz2' 

files = []
for year in years_to_download:
    # prepare strings
    url_of_data_file = baseurl%(year) # get the URL for the data file
    save_as_filename = 'Data/%d.csv.bz2'%(year) # save as this
    files += [save_as_filename] # save name of the compressed file
    
    # download file
    print('Downloading %s to %s'%(url_of_data_file, save_as_filename)) # progress update
    urllib.request.urlretrieve(url_of_data_file, save_as_filename) #execute download
    
print(files)

In order to work with the data files, they first need to be decompressed.

In [ ]:
import bz2

# Decompress all the files
for filename in files:
    # get file names
    filepath = filename
    newfilepath = filename[:-4]
    print('Decompressing', filepath,'to', newfilepath)
    
    # go through the decompressed chunks and write out to a decompressed file
    with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
        for data in iter(lambda : file.read(100 * 1024), b''):
            new_file.write(data)

Now that all of the files have been downloaded and decompressed, we can load a couple of them into memory just to make sure that they look correct before loading all of the files.

In [3]:
# Loading individual files in python
import pandas as pd
import numpy as np
import sys

total_length = 0
for year in [1987,1988]:
    # get file name of the csv
    # note that we can also load in the raw .bz2 file in python (or R) 
    # but the decompression step for these files sizes takes a huge performance hit
    csvfile = 'Data/%d.csv'%(year)
    print('loading', csvfile)
    sys.stdout.flush()
    
    # read the file and increment the lines count
    df = pd.read_csv(csvfile) # note that this is a big operation, especially since we just want the length
    # one way of making this shorter is to filter which columns we are loading
    
    total_length += len(df)

print('Answer from python:', total_length) 
df.head()

('loading', 'Data/1987.csv')
('loading', 'Data/1988.csv')
('Answer from python:', 6513922)


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1988,1,9,6,1348.0,1331,1458.0,1435,PI,942,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1988,1,10,7,1334.0,1331,1443.0,1435,PI,942,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1988,1,11,1,1446.0,1331,1553.0,1435,PI,942,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1988,1,12,2,1334.0,1331,1438.0,1435,PI,942,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1988,1,13,3,1341.0,1331,1503.0,1435,PI,942,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


The files appear to have downloaded correctly without being corrupted.  They are too large to be loaded into memory, so we will load them into one large GraphLab SFrame.  This is done by initializing an empty SFrame and then appending the data from each file to it one at a time in a for loop.  In order to be sure that the SFrame uses the correct data type for each variable, a list of data types will be passed in the column_type_hints parameter.  Once the SFrame has been populated, we will also save a compressed version of it in case we want to reload it directly without going through these initial steps at a later time.

In [2]:
import graphlab as gl
import time

In [3]:
column_hints=[int,int,int,int,int,int,int,int,str,int,str,int,int,int,int,int,str,str,int,int,int,int,str,int,int,int,int,int,int]

t = time.time()
sf = gl.SFrame()

for year in range(1987,2009):
    print 'read %d lines, reading next file %d.csv'%(sf.shape[0],year)
    sys.stdout.flush()
    sftmp = gl.SFrame.read_csv('Data/%d.csv'%(year),column_type_hints=column_hints)
    sf = sf.append(sftmp)

print 'It took %.2f seconds to concatenate the memory mapped file'%(time.time()-t)

t = time.time()
print 'Saving...',
sf.save('Data/sframe_directory')
print 'took %.2f seconds'%(time.time()-t),'Shape of SFrame is',sf.shape

This non-commercial license of GraphLab Create for academic use is assigned to zlbrown@smu.edu and will expire on August 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\xzach\AppData\Local\Temp\graphlab_server_1502166878.log.0


read 0 lines, reading next file 1987.csv


Read 540959 lines. Lines per second: 444650

Finished parsing file C:\Users\xzach\GitHub\SMU\MSDS 7333 - Quantifying the World\Unit 14 Case Study\Data\1987.csv

...

Read 3746137 lines. Lines per second: 577631

Read 6931041 lines. Lines per second: 586206

Finished parsing file C:\Users\xzach\GitHub\SMU\MSDS 7333 - Quantifying the World\Unit 14 Case Study\Data\2008.csv

Parsing completed. Parsed 7009728 lines in 11.973 secs.

It took 196.02 seconds to concatenate the memory mapped file
Saving... took 11.71 seconds Shape of SFrame is (123534969, 29)


We were able to concatenate the data from all of the csv files into one SFrame in a little over 3 minutes.  We then saved a compressed binary version of the SFrame in about 11 seconds.

In [1]:
# CONVENIENCE BLOCK FOR THOSE LOADING FROM DISK HERE

# If you have already run the notebook above and just want to load up the data
# then you can reload the SFrame here
import graphlab as gl

sf = gl.load_sframe('Data/sframe_directory')

This non-commercial license of GraphLab Create for academic use is assigned to zlbrown@smu.edu and will expire on August 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\xzach\AppData\Local\Temp\graphlab_server_1502325328.log.0


### Which airports are most likely to be delayed flying out of or into?

The first area we will analyze is delays by airport.  We will examine which origin airports are most likely to have departure delays and which destination airports are most likely to have arrival delays.

Since the data is to big to store in memory all at once, the split-apply-combine method will be used.  In graphlab this is done using the groupby function.  If we group by the origin airport and use the count aggregator, an SFrame containing the counts of departures for each airport.

In [2]:
%time sf_counts = sf.groupby('Origin', {'num_flights':gl.aggregate.COUNT()})
sf_counts

Wall time: 9.3 s


Origin,num_flights
ABY,8035
ATL,6100953
TEX,683
VLD,7300
CHS,222491
MKK,289
ROR,2228
VCT,2445
ITH,18397
LAW,18019


Next let's create new indicator columns for departure delays, arrival delays, and overall delays.  They will be set to 1 if there was a delay and 0 if there wasn't.

In [3]:
sf['DepDelayInd'] = (sf['DepTime'] - sf['CRSDepTime']) > 0
sf['ArrDelayInd'] = (sf['ArrTime'] - sf['CRSArrTime']) > 0
sf['DelayInd'] = sf['DepDelayInd'] | sf['ArrDelayInd']
sf['DepTime','CRSDepTime','DepDelayInd','ArrTime','CRSArrTime','ArrDelayInd','DelayInd']

DepTime,CRSDepTime,DepDelayInd,ArrTime,CRSArrTime,ArrDelayInd,DelayInd
741,730,1,912,849,1,1
729,730,0,903,849,1,1
741,730,1,918,849,1,1
729,730,0,847,849,0,0
749,730,1,922,849,1,1
728,730,0,848,849,0,0
728,730,0,852,849,1,1
731,730,1,902,849,1,1
744,730,1,908,849,1,1
729,730,0,851,849,1,1


The counts of departure delays can be calculated in a similar fashion to the counts of departures. We will still group by the origin and count the number of flights, but we will also have a second aggregator to sum the departure delays.  The percentage of delayed departures is then calculated by dividing the number of departure delays by the number of departures.

In [4]:
dep_delay_counts = sf.groupby('Origin', {'num_flights':gl.aggregate.COUNT()},
                                    {'dep_delays':gl.aggregate.SUM('DepDelayInd')})
dep_delay_counts['DepDelayPCT'] = dep_delay_counts['dep_delays'] / dep_delay_counts['num_flights']
dep_delay_counts = dep_delay_counts.sort('DepDelayPCT',ascending=False)
dep_delay_counts

Origin,num_flights,dep_delays,DepDelayPCT
BFI,1,1,1.0
CYS,2,2,1.0
MKC,1,1,1.0
FMN,3,3,1.0
BFF,1,1,1.0
OGD,6,5,0.833333333333
PIR,10,8,0.8
ADK,589,336,0.570458404075
SOP,319,176,0.551724137931
ATL,6100953,3149775,0.516275899847


The majority of the airports with the higest percentage of departure delays only have a handful of flights.  We will filter out airports with fewer than 100 departures in order to only look at airports with flights on at least a semi-regular basis.

In [5]:
dep_delay_counts[dep_delay_counts['num_flights'] > 100]

Origin,num_flights,dep_delays,DepDelayPCT
ADK,589,336,0.570458404075
SOP,319,176,0.551724137931
ATL,6100953,3149775,0.516275899847
PIT,2072303,1055547,0.509359393872
CLT,2546039,1290432,0.506839054704
OTH,515,257,0.499029126214
ORD,6597442,3275528,0.496484546586
DFW,5710980,2832680,0.496005939436
PHL,2165049,1055684,0.487602820999
UCA,4639,2124,0.457857296831


The airport in Adak, Alaska has the highest percentage of departure delays at 57%.  The remaining airports with the top ten departure delay percentages are SOP, ATL, PIT, CLT, OTH, ORD, DFW, PHL, and UCA.

Now, we will look at arrival delays.  We will need to group by destination to calculate this instead of grouping by origin.

In [6]:
arr_delay_counts = sf.groupby('Dest', {'num_flights':gl.aggregate.COUNT()},
                                    {'arr_delays':gl.aggregate.SUM('ArrDelayInd')})
arr_delay_counts['ArrDelayPCT'] = arr_delay_counts['arr_delays'] / arr_delay_counts['num_flights']
arr_delay_counts = arr_delay_counts.sort('ArrDelayPCT',ascending=False)
arr_delay_counts[arr_delay_counts['num_flights'] > 100]

Dest,num_flights,arr_delays,ArrDelayPCT
SLE,880,637,0.723863636364
OTH,517,367,0.709864603482
ANI,472,334,0.707627118644
SOP,317,211,0.665615141956
TTN,1671,1039,0.621783363256
GST,1878,1096,0.583599574015
HHH,1833,1057,0.576650300055
AKN,6001,3376,0.562572904516
KTN,51618,28363,0.549478863962
DLG,4940,2704,0.547368421053


Southwest Municipal Airport in Salem, Oregon has the highest percentage of delayed arrivals at a rate of about 72%.  The other nine airports in the top ten highest percentages of delayed arrivals are OTH, ANI, SOP, TTN, GST, HHH, AKN, KTN, and DLG.

### Which flights with same origin and destination are most likely to be delayed?

Next, we will determine which specific routes have the highest percentage of delays.  To do this, a new variable that contains both the origin and destination airports needs to be created.

In [7]:
sf['FlightRoute'] = sf['Origin'] + "-" + sf['Dest']
sf['Origin', 'Dest', 'FlightRoute']

Origin,Dest,FlightRoute
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO
SAN,SFO,SAN-SFO


We can now determine which flight paths have the most delays overall.  We will use the DelayInd variable that we created earlier to do this.  This vairable is set to 1 if the flight had a delayed departure or arrival.

In [8]:
delay_counts = sf.groupby('FlightRoute', {'num_flights':gl.aggregate.COUNT()},
                                    {'num_delays':gl.aggregate.SUM('DelayInd')})
delay_counts['NumDelayPCT'] = delay_counts['num_delays'] / delay_counts['num_flights']
delay_counts = delay_counts.sort('NumDelayPCT',ascending=False)
delay_counts[delay_counts['num_flights'] > 100]

FlightRoute,num_flights,num_delays,NumDelayPCT
MCO-DAB,163,157,0.963190184049
MIA-CHS,235,218,0.927659574468
MDT-RDU,158,146,0.924050632911
LGA-ROA,910,834,0.916483516484
RDU-ROA,306,280,0.915032679739
LGA-ICT,194,177,0.912371134021
EWR-UCA,204,186,0.911764705882
ANC-BRW,144,130,0.902777777778
RDU-MDT,161,145,0.900621118012
UCA-EWR,196,176,0.897959183673


The most delayed flight route is Orlando International Airport to Daytona Beach International Airport.  157 of the 163 flights on this route had either a delayed departure or arrival.  That is about 96% of the total flights on that route.  The other 9 of the top ten most delayed flight paths are MIA-CHS, MDT-RDU, LGA-ROA, RDU-ROA, LGA-ICT, EWR-UCA, ANC-BRW, RDU-MDT, and UCA-EWR.

### Regress how delayed a flight will be before it is delayed

Next, we will use linear regression to try to determine how delayed a flight will be.  

In order to simplify things, we will create two new variables that contain the floor of the departure and arrival times to the nearest hour.

In [9]:
from math import floor
sf['DepTimeByHour'] = sf['CRSDepTime'].apply(lambda x: floor(x/100),dtype=int)
sf['DepTimeByHour']
sf['ArrTimeByHour'] = sf['CRSArrTime'].apply(lambda x: floor(x/100),dtype=int)
sf['ArrTimeByHour']

dtype: int
Rows: 123534969
[8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 8L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 10L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 16L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 17L, 8L, 9L, 9L, 9L, 9L, 9L, ... ]

Some records have the departure hour set to 0 and some have it set to 24.  Since these are the same time, we will change all of the values of 24 to 0.

In [10]:
sf['DepTimeByHour'] = sf['DepTimeByHour'].apply(lambda x: 0 if x==24 else x)
sf['ArrTimeByHour'] = sf['ArrTimeByHour'].apply(lambda x: 0 if x==24 else x)

In order to keep the original SFrame intact, we will create a new SFrame that only contains the features that we might use in a linear regression model.  This does not include any features that would be unknown before the flight (such as delay times and cancellations).  We also want to create a new column that contains the age of each specific plane.  We will use the tail number to do this.  Since the tail numbers were not recorded before 1995, we will only use records from after 1994.

In [11]:
sf_tmp = sf[['DepDelay','TailNum','Year','Month','DayofMonth','DayOfWeek','DepTimeByHour','ArrTimeByHour','UniqueCarrier','Origin','Dest','CRSElapsedTime','Distance','FlightRoute']][sf['Year']>1994]

Now we will calculate the age of the planes at the time of flight.  To do this, we first create a variable named FlightAge that contains the number of months since the year 0 at the time of the flight.

Each plane has a unique tail number that does not change.  For each tail number, we find the minimum FlightAge and store these results in a new SFrame called sf_min_ages.

In [12]:
import time
sf_tmp['FlightAge'] = 12*sf_tmp['Year']+sf_tmp['Month']-1

t = time.time()
sf_min_ages = sf_tmp[['TailNum','FlightAge']].groupby('TailNum',{'FirstFlight':gl.aggregate.MIN('FlightAge')})
print 'Took %.2f seconds to run'%(time.time()-t)

Took 8.14 seconds to run


Now we join the original SFrame with the new SFrame that contains the minimum ages of the planes.

In [13]:
%time sf_fewcols = sf_tmp[['TailNum','FlightAge']].join(sf_min_ages,on='TailNum',how='left') # long operation

Wall time: 37.3 s


Finally, the time since the first recorded flight for each tail number is calculated by subtracting the FirstFlight variable from the FlightAge variable.

In [14]:
sf_tmp['Age'] = sf_fewcols['FlightAge']-sf_fewcols['FirstFlight']

Now that we have created some additional features, the data can be prepared for cross validation.  The SFrame is currently ordered by year.  We will need to shuffle the SFrame in order to produce random folds.

In [15]:
sf_shuffled = gl.cross_validation.shuffle(sf_tmp.dropna())

Now that the data has been shuffled, we can create the graphlab cross validation object.  For the purposes of this exercise, 5 fold cross validation will be used.

In [16]:
folds = gl.cross_validation.KFold(sf_shuffled, 5)

The cross validation object has been created and we are now ready to run linear regression.  We are interested in trying to determine how long a flight will be delayed before it is delayed.  Therefore, the target variable will be DepDelay.  This variable contains the number of minutes that a flight's departure was delayed.

The features that will be supplied for possible inclusion in the model are the month, day of week, hour of departure time, hour of arrival time, flight route, scheduled flight time in minutes, and age of the plane.  Any variables that are calculated after a flight takes off (such as taxi times and weather delays) will not be included.  

The graphlab linear_regression function contains a solver parameter that can perform feature selection automatically.  There are three different solvers that can be used - Newton-Raphson, limited memory BFGS, and accelerated gradient descent.  We will leave the solver parameter set to the default of "auto."  This will automatically determine the optimal solver for the data and model parameters.

In [25]:
params = dict([('target', 'DepDelay'),('features',['Month','DayOfWeek','DepTimeByHour','ArrTimeByHour','FlightRoute','CRSElapsedTime','Age'])])
job = gl.cross_validation.cross_val_score(folds,
                                         gl.linear_regression.create,
                                         params)

print(job.get_results())

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Cross-Validation-Aug-09-2017-21-26-53-318000--1815483' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Cross-Validation-Aug-09-2017-21-26-53-318000--1815483' scheduled.
[INFO] graphlab.deploy._job: Waiting for job to finish, this may take quite a while.
[INFO] graphlab.deploy._job: You may CTRL-C to stop this command and it will not cancel your job.


OrderedDict([('models', [Class                          : LinearRegression

Schema
------
Number of coefficients         : 7547
Number of examples             : 67518053
Number of feature columns      : 7
Number of unpacked features    : 7

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : TERMINATED: Iteration limit reached.
Training time (sec)            : 292.8456

Settings
--------
Residual sum of squares        : 62689339900.9
Training RMSE                  : 30.471

Highest Positive Coefficients
-----------------------------
FlightRoute[CMI-SPI]           : 574.6707
FlightRoute[SUX-OMA]           : 410.5607
FlightRoute[BIS-FAR]           : 352.5635
FlightRoute[MCI-SGF]           : 344.2863
FlightRoute[BFL-SBA]           : 320.9974

Lowest Negative Coefficients
-----------------------

After some time, the cross validated linear regression returned successfully.  

The limited memory BFGS solver was chosen and left all 7 features in the model.

The model appears to be fairly accurate with an average validation root mean square error of 30.5.  The average training root mean square error is also 305, so the model appears to generalize well and does not appear to be overfit to the training data.

The flight route seems to be the feature that is most highly correlated with the length of departure delay, as the 5 most positively and negatively correlated features for each fold are all dummy variables created for individual flight routes.

## Conclusions and Future Work

With the ever increasing availability of large datasets, new challenges have arisen related to the ability to actually process the data.  The aim of this case study was to determine if it would be feasible to perform some standard data exploration and rudimentary machine learning on a data set that is too large to fit into memory on an average machine.  Using the Turi GraphLab library, this was able to be accomplished fairly easily.  Although the data set is large, most of the operations were performed in times that were not incredibly excessive.

As is the case with most projects, there is room for future work with this case study in several areas.  First of all, the data itself could be explored further.  We only just scratched the surface focusing specifically on delayed flights.  Techniques other than linear regression could be explored for predicting the length of delays.  There is also room for analysis in other areas (service time of individual planes, weather related delays, etc.).  In addition to further analysis on the data itself, different libraries/tools for out of core memory operations could be examined.  Perhaps there are better alternatives to GraphLab that could be used.